In [1]:
pip install -U pygame --user

Requirement already up-to-date: pygame in /Users/rishichalla/.local/lib/python3.8/site-packages (2.0.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install qiskit

In [3]:
import pygame, sys
from pygame.locals import *
import io
import math
import pygame.math as pyMath
from urllib.request import urlopen
from random import randint
import qiskit
from qiskit import *

#import menu
%run Credits.ipynb
%run Controls.ipynb
%run level_select.ipynb

#import Levels
%run ../Levels/Level_0.ipynb
%run ../Levels/Level_Entanglement.ipynb
#%run ../Levels/Level_1.ipynb

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#conda install -c cogsci pygame

In [5]:
pygame.init()

(7, 0)

In [6]:
screen = pygame.display.set_mode((800, 600))

In [7]:
running = True
isAlive = True

In [8]:
#Sprites
icon = pygame.image.load('../Images/duck.png')

In [9]:
#Backgrounds
## Menu
load_bg = pygame.image.load("../Images/bg.jpg")
blackHole_img = pygame.image.load("../Images/blackHole.jpg")
blackHole_bg = pygame.transform.scale(blackHole_img, (800, 600))
bg = pygame.transform.scale(load_bg, (800, 600))
## Level 0: Tutorial
level0_load = pygame.image.load("../Images/level0.png")
level0 = pygame.transform.scale(level0_load, (800, 600))
## Level 2: Entanglement
level_entanglement = pygame.image.load("../Images/entanglementBg.png")
entanglement_bg = pygame.transform.scale(level_entanglement, (800, 600))

In [10]:
#Global Variables
pygame.display.set_caption("Qwack")
meme = pygame.image.load('../Images/qwackMeme.jpg')
pygame.display.set_icon(meme)

fontTitle = pygame.font.SysFont("Comic", 125)
fontSubHeader = pygame.font.SysFont(None, 50)
text = pygame.font.SysFont(None, 25)

mainClock = pygame.time.Clock()

display = True
click = False
themeColor = (255,255,0)



In [11]:
def draw_text(text, font, color, surface, x, y):
    textObj = font.render(text,1,color)
    textRect = textObj.get_rect()
    textRect.topleft = (x,y)
    surface.blit(textObj, textRect)
    return textRect

In [12]:
#icon
global iconX, iconY, velocity
iconX = randint(10, 768)
iconY = randint(10, 568)
velocity = [2.5,1.5]

In [13]:
def Icon(sprite, x,y):
    screen.blit(sprite, (x, y))
    rect = pygame.Rect(x, y, 32, 32)
    return rect

In [14]:
def Distance(x2, y2):
    hole = pyMath.Vector2(400, 300)
    dist = hole.distance_to(pyMath.Vector2(x2, y2))
    return dist

In [15]:
def Magnitude(x, y):
    return (x ** 2 + y ** 2) ** 0.5

In [16]:
def changeGrav(duckHole, grav):
    newDuckHole = pyMath.Vector2(duckHole.x/grav, duckHole.y/grav)
    return pyMath.Vector2(newDuckHole.x+velocity[0], newDuckHole.y+velocity[1])

In [17]:
def main_menu(iconX, iconY, isAlive):
    blackHole_rect = pygame.Rect(382, 287, 32, 32)  # blackHole hitbox
    duck_button = pygame.Rect(0,0,32,32) # duck hitbox
    hole = pyMath.Vector2(400, 300) #blackHole x, y position in vector
    blackHole = False # if True, blackHole code runs
    previous_coord = pyMath.Vector2(0,0) # storing inital duck coordinates before movement 
    dist = 0 # distance value init
    
    while True:
        
        # Setup
        previous_coord.x = iconX  # setting x and y values
        previous_coord.y = iconY

        if blackHole == False: 
            screen.blit(bg, (0,0)) # regular background image render
        else: 
            screen.blit(blackHole_bg, (0,0)) # black hole background image render
        mx, my = pygame.mouse.get_pos()   # get mouse position for interaction    
            
        # blackHole gravity    
                
        dist = Distance(iconX, iconY) # call distance function to calculate black hole gravity 
            
        # initial velocity (no black hole)
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
                
        
        if blackHole: # velocity with black hole
            duck_to_hole = pyMath.Vector2((400-previous_coord.x), (300-previous_coord.y)) # vector from initial duck position to black hole
            resultant = pyMath.Vector2(duck_to_hole.x + velocity[0], duck_to_hole.y + velocity[1]) # resultant vector from duck to hole and velocity vectors

            # calculating varying gravity from duck Distance to black hole
            if (dist/175) > .99: # gravity influence starting point 
                iconX += velocity[0]
                iconY += velocity[1]
            elif (dist/175) >= .75 and (dist/175) <= .99: # first variation 
                resultant = changeGrav(duck_to_hole, 50) # call changeGrav function to add the gravity influence to duck vector
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1: 
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            elif (dist/175) >= .50 and (dist/175) < .75: # second variation 
                resultant = changeGrav(duck_to_hole, 40) # gravity is a little stronger; smaller the number, the stronger the gravity
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1:
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            elif (dist/175) >= .25 and (dist/175) < .5: # third variation 
                resultant = changeGrav(duck_to_hole, 20)
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1:
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            elif (dist/175) < .25:
                resultant = changeGrav(duck_to_hole, 1) # final variation, pulls duck in all the way (event horizion)
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1:
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            if duck_button.colliderect(blackHole_rect): # collision detection between the duck and black hole to trigger duck vanishing
                isAlive = False
                blackHole = False
                
        else:  # if no black hole
            iconX += velocity[0]
            iconY += velocity[1]
        
#         duck_to_hole_mag = Magnitude(duck_to_hole.x, duck_to_hole.y)
#         prev_to_cur_mag = Magnitude(velocity[0],velocity[1])
        
        # determines whether to show duck or not
        if isAlive:
            duck_button = Icon(icon, iconX, iconY)
        
        # Text
        home_button = draw_text("Qwack", fontTitle, themeColor, screen, 260, 15)
        start_game_button = draw_text("Start Game", fontSubHeader, themeColor, screen, 75, 150)
        controls_button = draw_text("Controls", fontSubHeader, themeColor, screen, 75, 225)
        credits_button = draw_text("Credits", fontSubHeader, themeColor, screen, 75, 300)
        
        # Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT: #if user wants to exit game
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN: 
                if event.key == K_ESCAPE: #when user presses esc it leaves game
                    pygame.quit()
                    sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1: # checking for clicks
                    click = True
                    #print(mx, my)
        
        # Checking for user clicks on buttons
        if home_button.collidepoint((mx, my)): # if user clicks on 'Qwack' title, returns to menu page
            if click:
                main_menu(iconX, iconY, isAlive)            
        if start_game_button.collidepoint((mx, my)): # if user clicks on 'Level Select', goes to level select page
            if click:
                level_select(iconX, iconY, isAlive)
        if controls_button.collidepoint((mx, my)): # if user clicks on 'Controls', goes to controls page
            if click:
                Controls(iconX, iconY, isAlive)
        if credits_button.collidepoint((mx, my)): # if user clicks on 'Credits', goes to credits page
            if click:
                Credits(iconX, iconY, isAlive)
        if duck_button.collidepoint((mx, my)): # if user clicks on floating duck icon, black hole appears
            if click:
                blackHole = True
                
        
        pygame.display.update() # update the game with changes from this loop
        mainClock.tick(60) # FPS

In [ ]:
# Game Loop
main_menu(iconX, iconY, isAlive)

<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>
<rect(100, 450, 32, 32)>


<rect(370, 0, 32, 32)>
<rect(365, 0, 32, 32)>
<rect(360, 0, 32, 32)>
<rect(355, 0, 32, 32)>
<rect(350, 0, 32, 32)>
<rect(345, 5, 32, 32)>
<rect(340, 10, 32, 32)>
<rect(335, 15, 32, 32)>
<rect(335, 20, 32, 32)>
<rect(335, 25, 32, 32)>
<rect(335, 30, 32, 32)>
<rect(335, 35, 32, 32)>
<rect(335, 40, 32, 32)>
<rect(335, 45, 32, 32)>
<rect(335, 50, 32, 32)>
<rect(340, 55, 32, 32)>
<rect(345, 60, 32, 32)>
<rect(350, 65, 32, 32)>
<rect(355, 70, 32, 32)>
<rect(360, 75, 32, 32)>
<rect(365, 80, 32, 32)>
<rect(370, 85, 32, 32)>
<rect(375, 90, 32, 32)>
<rect(380, 95, 32, 32)>
<rect(385, 95, 32, 32)>
<rect(390, 95, 32, 32)>
<rect(395, 95, 32, 32)>
<rect(400, 95, 32, 32)>
<rect(405, 95, 32, 32)>
<rect(410, 95, 32, 32)>
<rect(415, 95, 32, 32)>
<rect(420, 95, 32, 32)>
<rect(425, 95, 32, 32)>
<rect(430, 95, 32, 32)>
<rect(435, 95, 32, 32)>
<rect(440, 95, 32, 32)>
<rect(445, 100, 32, 32)>
<rect(450, 105, 32, 32)>
<rect(455, 110, 32, 32)>
<rect(460, 115, 32, 32)>
<rect(460, 120, 32, 32)>
<rect(460, 125, 3